In [1]:
import os    # For file paths
import pandas as pd 
from pandas import isnull

In [2]:
# Master.csv
master = pd.read_csv(os.path.join("..", "data", "Master.csv"))
master.head()

,playerID,coachID,hofID,firstName,lastName,nameNote,nameGiven,nameNick,height,weight,...,birthDay,birthCountry,birthState,birthCity,deathYear,deathMon,deathDay,deathCountry,deathState,deathCity
0,aaltoan01,NaN,NaN,Antti,Aalto,NaN,Antti,NaN,73.0,210.0,...,4.0,Finland,NaN,Lappeenranta,NaN,NaN,NaN,NaN,NaN,NaN
1,abbeybr01,NaN,NaN,Bruce,Abbey,NaN,Bruce,NaN,73.0,185.0,...,18.0,Canada,ON,Toronto,NaN,NaN,NaN,NaN,NaN,NaN
2,abbotge01,NaN,NaN,George,Abbott,NaN,George Henry,Preacher,67.0,153.0,...,3.0,Canada,ON,Synenham,NaN,NaN,NaN,NaN,NaN,NaN
3,abbotre01,NaN,NaN,Reg,Abbott,NaN,Reginald Stewart,NaN,71.0,164.0,...,4.0,Canada,MB,Winnipeg,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelju01,NaN,NaN,Justin,Abdelkader,NaN,NaN,NaN,73.0,195.0,...,25.0,USA,MI,Muskegon,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
master.shape

In [ ]:
master.columns

In [ ]:
# Braces let us structure the code in a more readable way
(master["playerID"]
     .pipe(isnull)
     .value_counts())
    
# This is an alternative to:
isnull(master["playerID"]).value_counts()

In [ ]:
master_orig = master.copy()
master = master.dropna(subset=["playerID"])
master.shape

In [ ]:
master.columns

In [ ]:
master = master.dropna(subset=["firstNHL", "lastNHL"], how="all")

In [ ]:
master = master.loc[master["lastNHL"] >= 1980]
master.shape

In [ ]:
master.columns

In [ ]:
columns_to_keep = ["playerID", "firstName", "lastName",
                   "pos", "birthYear", "birthMon", "birthDay",
                   "birthCountry", "birthState", "birthCity"]

master[columns_to_keep].head()
master.filter(columns_to_keep).head()

In [ ]:
master = master.filter(regex="(playerID|pos|^birth)|(Name$)")
master.columns

In [ ]:
def mem_mib(df):
    print("{0:.2f} MiB".format(
        df.memory_usage().sum() / (1024 * 1024)
    ))
    
mem_mib(master)
mem_mib(master_orig)

In [ ]:
master["pos"].value_counts()

In [ ]:
pd.Categorical(master["pos"])

In [ ]:
def make_categorical(df, col_name):
    df.loc[:, col_name] = pd.Categorical(df[col_name]) 

In [ ]:
make_categorical(master, "pos")
make_categorical(master, "birthCountry")
make_categorical(master, "birthState")
mem_mib(master)

In [ ]:
master.head()

In [ ]:
master = master.set_index("playerID")
master.head()

In [ ]:
master.to_pickle(os.path.join("..", "master.pickle"))

## Scoring.csv 

In [ ]:
scoring = pd.read_csv(os.path.join("..", "data", "Scoring.csv"))
mem_mib(scoring)
scoring.shape

In [ ]:
scoring.columns

In [ ]:
def recent_nhl_only(df):
    return df[(df["lgID"] == "NHL") & (df["year"] >= 1980)]

scoring = recent_nhl_only(scoring)
scoring.shape      

In [ ]:
scoring.columns

In [ ]:
scoring = scoring.filter(regex="^(?!(Post|PP|SH)).*")
scoring.columns

In [ ]:
scoring = scoring.iloc[:, [0, 1, 3, 6, 7, 8, 9, 14]]
scoring.columns

In [ ]:
make_categorical(scoring, "tmID")
scoring.head()

In [ ]:
scoring.reset_index().head()

In [ ]:
scoring = scoring.reset_index(drop=True)
# Alternatively:
scoring.reset_index(drop=True, inplace=True)
scoring.head()

In [ ]:
scoring.to_pickle(os.path.join("..", "scoring.pickle"))

## Teams.csv 

In [ ]:
teams = pd.read_csv(os.path.join("..", "data", "Teams.csv"))
teams.shape

In [ ]:
teams.columns

In [ ]:
teams = recent_nhl_only(teams)
teams = teams[["year", "tmID", "name"]]
teams.head()

In [ ]:
teams.nunique()

In [ ]:
make_categorical(teams, "tmID")

In [ ]:
teams.to_pickle(os.path.join("..", "teams.pickle"))

## TeamSplits.csv 

In [ ]:
team_splits = pd.read_csv(os.path.join("..", "data", "TeamSplits.csv"))
team_splits.shape

In [ ]:
team_splits.columns

In [ ]:
team_splits = recent_nhl_only(team_splits)

In [ ]:
cols_to_drop = team_splits.columns[3:11]
team_splits = team_splits.drop(columns=cols_to_drop)
team_splits.columns
# some_data_frame.drop(rows=row_labels) <- to drop rows

In [ ]:
team_splits = team_splits.drop(columns="lgID")

In [ ]:
make_categorical(team_splits, "tmID")
team_splits.to_pickle(os.path.join("..", "team_splits.pickle"))